In [ ]:
# ================================================================
# PART 2: ASTHMA ONSET PREDICTION
# ================================================================

import warnings
import numpy as np
import pandas as pd
import pickle
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.utils.class_weight import compute_sample_weight
from sklearn.metrics import (roc_auc_score, classification_report, confusion_matrix, 
                             accuracy_score, precision_score, recall_score, f1_score,
                             precision_recall_curve, roc_curve, auc)
from sklearn.model_selection import StratifiedKFold
import xgboost as xgb
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

warnings.filterwarnings('ignore')
sns.set_style("whitegrid")

# ============================================================================
# LOAD DATA & CREATE ONSET LABELS
# ============================================================================

train_original = pd.read_csv("train_cleaned.csv")
test_original = pd.read_csv("test_cleaned.csv")

with open("weight_info.pkl", "rb") as f:
    weight_info = pickle.load(f)

# Filter for asthmatic patients only
train_asthma = train_original[train_original['Asthmatic'] == 1].copy()
test_asthma = test_original[test_original['Asthmatic'] == 1].copy()

# Create onset labels: 0=Childhood (<18), 1=Adult (≥18)
def create_onset_label(row):
    age_at_diagnosis = row.get('Age when first had asthma', np.nan)
    if pd.isna(age_at_diagnosis):
        return np.nan
    return 0 if age_at_diagnosis < 18 else 1

train_asthma['Onset_Type'] = train_asthma.apply(create_onset_label, axis=1)
test_asthma['Onset_Type'] = test_asthma.apply(create_onset_label, axis=1)

# Remove cases with missing onset info
train_asthma = train_asthma[train_asthma['Onset_Type'].notna()].copy()
test_asthma = test_asthma[test_asthma['Onset_Type'].notna()].copy()

train_childhood = (train_asthma['Onset_Type'] == 0).sum()
train_adult = (train_asthma['Onset_Type'] == 1).sum()
test_childhood = (test_asthma['Onset_Type'] == 0).sum()
test_adult = (test_asthma['Onset_Type'] == 1).sum()

print(f"Train: Childhood={train_childhood}, Adult={train_adult}")
print(f"Test: Childhood={test_childhood}, Adult={test_adult}")


In [ ]:
# ============================================================================
# FEATURE SELECTION
# ============================================================================

exclude_cols = [
    'SEQN',
    'Age',  # CRITICAL: Age is confounded with time since diagnosis
    'Full sample 2 year interview weight',
    'Full sample 2 year MEC exam weight',
    'Asthmatic',
    'Age when first had asthma',
    'Still have asthma',
    'Asthma attack in past year',
    'Emergency care visit for asthma/past yr',
    'Onset_Type'
]

features = [col for col in train_asthma.columns if col not in exclude_cols]

# ============================================================================
# PREPARE DATA
# ============================================================================

X_train = train_asthma[features].copy()
y_train = train_asthma['Onset_Type'].copy()
X_test = test_asthma[features].copy()
y_test = test_asthma['Onset_Type'].copy()

train_weights = train_asthma[weight_info['interview_weight']].values
test_weights = test_asthma[weight_info['interview_weight']].values

# Handle missing values
missing_count = X_train.isnull().sum().sum() + X_test.isnull().sum().sum()
if missing_count > 0:
    for col in X_train.columns:
        if X_train[col].isnull().any():
            median_val = X_train[col].median()
            X_train[col] = X_train[col].fillna(median_val)
            X_test[col] = X_test[col].fillna(median_val)

# Encode categorical variables
categorical_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
if categorical_cols:
    X_combined = pd.concat([X_train, X_test], axis=0, keys=['train', 'test'])
    X_combined = pd.get_dummies(X_combined, columns=categorical_cols, drop_first=True)
    X_train = X_combined.loc['train'].copy()
    X_test = X_combined.loc['test'].copy()

X_train = X_train.astype(float)
X_test = X_test.astype(float)


In [ ]:
# ============================================================================
# CROSS-VALIDATION SETUP
# ============================================================================

n_folds = 5
skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

cv_results = {
    'LR': {'auc': [], 'accuracy': [], 'f1': []},
    'XGB': {'auc': [], 'accuracy': [], 'f1': []},
    'RF': {'auc': [], 'accuracy': [], 'f1': []},
    'ET': {'auc': [], 'accuracy': [], 'f1': []}
}

# ============================================================================
# CROSS-VALIDATION LOOP
# ============================================================================

for fold, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train), 1):
    print(f"\nFold {fold}/{n_folds}")
    
    # Split data
    X_tr, X_val = X_train.iloc[train_idx], X_train.iloc[val_idx]
    y_tr, y_val = y_train.iloc[train_idx], y_train.iloc[val_idx]
    w_tr, w_val = train_weights[train_idx], train_weights[val_idx]
    
    # Balance classes
    class_weights = compute_sample_weight('balanced', y_tr)
    w_tr_balanced = w_tr * class_weights
    w_tr_balanced = w_tr_balanced * (w_tr.sum() / w_tr_balanced.sum())
    
    # Standardize
    scaler = StandardScaler()
    X_tr_scaled = scaler.fit_transform(X_tr)
    X_val_scaled = scaler.transform(X_val)
    
    # Train and evaluate each model
    
    # Logistic Regression
    lr = LogisticRegression(C=1.0, solver='lbfgs', max_iter=1000, random_state=42)
    lr.fit(X_tr_scaled, y_tr, sample_weight=w_tr_balanced)
    lr_proba = lr.predict_proba(X_val_scaled)[:, 1]
    lr_pred = (lr_proba >= 0.5).astype(int)
    
    cv_results['LR']['auc'].append(roc_auc_score(y_val, lr_proba, sample_weight=w_val))
    cv_results['LR']['accuracy'].append(accuracy_score(y_val, lr_pred, sample_weight=w_val))
    cv_results['LR']['f1'].append(f1_score(y_val, lr_pred, sample_weight=w_val, zero_division=0))
    
    # XGBoost
    fold_childhood = (y_tr == 0).sum()
    fold_adult = (y_tr == 1).sum()
    scale_pos_weight = fold_adult / fold_childhood if fold_childhood > 0 else 1.0
    
    xgb_model = xgb.XGBClassifier(
        n_estimators=150, max_depth=4, learning_rate=0.05,
        subsample=0.8, colsample_bytree=0.8,
        scale_pos_weight=scale_pos_weight,
        random_state=42, eval_metric='auc', use_label_encoder=False
    )
    xgb_model.fit(X_tr, y_tr, sample_weight=w_tr_balanced, verbose=False)
    xgb_proba = xgb_model.predict_proba(X_val)[:, 1]
    xgb_pred = (xgb_proba >= 0.5).astype(int)
    
    cv_results['XGB']['auc'].append(roc_auc_score(y_val, xgb_proba, sample_weight=w_val))
    cv_results['XGB']['accuracy'].append(accuracy_score(y_val, xgb_pred, sample_weight=w_val))
    cv_results['XGB']['f1'].append(f1_score(y_val, xgb_pred, sample_weight=w_val, zero_division=0))
    
    # Random Forest
    rf = RandomForestClassifier(
        n_estimators=200, max_depth=8, min_samples_split=20,
        min_samples_leaf=10, random_state=42, n_jobs=-1
    )
    rf.fit(X_tr, y_tr, sample_weight=w_tr_balanced)
    rf_proba = rf.predict_proba(X_val)[:, 1]
    rf_pred = (rf_proba >= 0.5).astype(int)
    
    cv_results['RF']['auc'].append(roc_auc_score(y_val, rf_proba, sample_weight=w_val))
    cv_results['RF']['accuracy'].append(accuracy_score(y_val, rf_pred, sample_weight=w_val))
    cv_results['RF']['f1'].append(f1_score(y_val, rf_pred, sample_weight=w_val, zero_division=0))
    
    # Extra Trees
    et = ExtraTreesClassifier(
        n_estimators=200, max_depth=12, min_samples_split=15,
        min_samples_leaf=8, random_state=42, n_jobs=-1
    )
    et.fit(X_tr, y_tr, sample_weight=w_tr_balanced)
    et_proba = et.predict_proba(X_val)[:, 1]
    et_pred = (et_proba >= 0.5).astype(int)
    
    cv_results['ET']['auc'].append(roc_auc_score(y_val, et_proba, sample_weight=w_val))
    cv_results['ET']['accuracy'].append(accuracy_score(y_val, et_pred, sample_weight=w_val))
    cv_results['ET']['f1'].append(f1_score(y_val, et_pred, sample_weight=w_val, zero_division=0))

# ============================================================================
# SUMMARIZE CV RESULTS
# ============================================================================

cv_summary = []
for model_name, metrics in cv_results.items():
    cv_summary.append({
        'Model': model_name,
        'CV_AUC_Mean': np.mean(metrics['auc']),
        'CV_AUC_Std': np.std(metrics['auc']),
        'CV_Accuracy_Mean': np.mean(metrics['accuracy']),
        'CV_Accuracy_Std': np.std(metrics['accuracy']),
        'CV_F1_Mean': np.mean(metrics['f1']),
        'CV_F1_Std': np.std(metrics['f1'])
    })

cv_summary_df = pd.DataFrame(cv_summary)
cv_summary_df.to_csv('onset_cv_results.csv', index=False)


In [ ]:
# ============================================================================
# TRAIN FINAL MODELS ON FULL TRAINING SET
# ============================================================================

print("\n\nTraining final models on full training set...")

class_weights = compute_sample_weight('balanced', y_train)
train_weights_balanced = train_weights * class_weights
train_weights_balanced = train_weights_balanced * (train_weights.sum() / train_weights_balanced.sum())

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

models = {}
predictions = {}

# Logistic Regression
lr = LogisticRegression(C=1.0, solver='lbfgs', max_iter=1000, random_state=42)
lr.fit(X_train_scaled, y_train, sample_weight=train_weights_balanced)
predictions['LR'] = lr.predict_proba(X_test_scaled)[:, 1]
models['LR'] = lr

# XGBoost
scale_pos_weight = train_adult / train_childhood if train_childhood > 0 else 1.0
xgb_model = xgb.XGBClassifier(
    n_estimators=150, max_depth=4, learning_rate=0.05,
    subsample=0.8, colsample_bytree=0.8,
    scale_pos_weight=scale_pos_weight,
    random_state=42, eval_metric='auc', use_label_encoder=False
)
xgb_model.fit(X_train, y_train, sample_weight=train_weights_balanced, verbose=False)
predictions['XGB'] = xgb_model.predict_proba(X_test)[:, 1]
models['XGB'] = xgb_model

# Random Forest
rf = RandomForestClassifier(
    n_estimators=200, max_depth=8, min_samples_split=20,
    min_samples_leaf=10, random_state=42, n_jobs=-1
)
rf.fit(X_train, y_train, sample_weight=train_weights_balanced)
predictions['RF'] = rf.predict_proba(X_test)[:, 1]
models['RF'] = rf

# Extra Trees
et = ExtraTreesClassifier(
    n_estimators=200, max_depth=12, min_samples_split=15,
    min_samples_leaf=8, random_state=42, n_jobs=-1
)
et.fit(X_train, y_train, sample_weight=train_weights_balanced)
predictions['ET'] = et.predict_proba(X_test)[:, 1]
models['ET'] = et

# ============================================================================
# EVALUATE ON TEST SET
# ============================================================================

results = []
for name, proba in predictions.items():
    pred = (proba >= 0.5).astype(int)
    
    roc_auc = roc_auc_score(y_test, proba, sample_weight=test_weights)
    acc = accuracy_score(y_test, pred, sample_weight=test_weights)
    precision = precision_score(y_test, pred, sample_weight=test_weights, zero_division=0)
    recall = recall_score(y_test, pred, sample_weight=test_weights, zero_division=0)
    f1 = f1_score(y_test, pred, sample_weight=test_weights, zero_division=0)
    
    precision_childhood = precision_score(y_test, pred, pos_label=0, sample_weight=test_weights, zero_division=0)
    recall_childhood = recall_score(y_test, pred, pos_label=0, sample_weight=test_weights, zero_division=0)
    f1_childhood = f1_score(y_test, pred, pos_label=0, sample_weight=test_weights, zero_division=0)
    
    results.append({
        'Model': name,
        'Test_AUC': roc_auc,
        'Test_Accuracy': acc,
        'Test_Precision_Adult': precision,
        'Test_Recall_Adult': recall,
        'Test_F1_Adult': f1,
        'Test_Precision_Childhood': precision_childhood,
        'Test_Recall_Childhood': recall_childhood,
        'Test_F1_Childhood': f1_childhood
    })

results_df = pd.DataFrame(results)

# Merge with CV results
final_results = cv_summary_df.merge(results_df, on='Model')
final_results.to_csv('onset_detailed_metrics_with_cv.csv', index=False)


# Get best model
best_idx = results_df['Test_AUC'].idxmax()
best_model_name = results_df.loc[best_idx, 'Model']
best_proba = predictions[best_model_name]
best_pred = (best_proba >= 0.5).astype(int)

# Save confusion matrix
cm = confusion_matrix(y_test, best_pred, sample_weight=test_weights)
cm_df = pd.DataFrame(
    cm,
    index=['Actual_Childhood', 'Actual_Adult'],
    columns=['Predicted_Childhood', 'Predicted_Adult']
)
cm_df.to_csv('onset_confusion_matrix.csv')

# Save classification report
report_dict = classification_report(
    y_test, best_pred, 
    sample_weight=test_weights,
    target_names=['Childhood_Onset', 'Adult_Onset'],
    output_dict=True,
    zero_division=0
)
report_df = pd.DataFrame(report_dict).transpose()
report_df.to_csv('onset_classification_report.csv')


In [ ]:
# ============================================================================
# FEATURE IMPORTANCE
# ============================================================================

if best_model_name == 'LR':
    importance = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': np.abs(models['LR'].coef_[0]),
        'Coefficient': models['LR'].coef_[0],
        'Direction': ['Adult↑' if c > 0 else 'Child↑' for c in models['LR'].coef_[0]]
    }).sort_values('Importance', ascending=False)
else:
    importance = pd.DataFrame({
        'Feature': X_train.columns,
        'Importance': models[best_model_name].feature_importances_
    }).sort_values('Importance', ascending=False)

importance.to_csv('onset_feature_importance.csv', index=False)

# ============================================================================
# SAVE MODELS
# ============================================================================

joblib.dump(models['LR'], 'onset_lr_model.pkl')
joblib.dump(models['XGB'], 'onset_xgb_model.pkl')
joblib.dump(models['RF'], 'onset_rf_model.pkl')
joblib.dump(models['ET'], 'onset_et_model.pkl')
joblib.dump(scaler, 'onset_scaler.pkl')

predictions_df = pd.DataFrame({
    'y_true': y_test,
    'lr_proba': predictions['LR'],
    'xgb_proba': predictions['XGB'],
    'rf_proba': predictions['RF'],
    'et_proba': predictions['ET'],
    'best_prediction': best_pred,
    'test_weight': test_weights
})
predictions_df.to_csv('onset_predictions.csv', index=False)

# Save CV results per fold for visualization
cv_folds_data = []
for model_name, metrics in cv_results.items():
    for fold_idx, (auc_val, acc_val, f1_val) in enumerate(zip(metrics['auc'], metrics['accuracy'], metrics['f1']), 1):
        cv_folds_data.append({
            'Model': model_name,
            'Fold': fold_idx,
            'AUC': auc_val,
            'Accuracy': acc_val,
            'F1': f1_val
        })

cv_folds_df = pd.DataFrame(cv_folds_data)
cv_folds_df.to_csv('onset_cv_folds.csv', index=False)

In [ ]:
# ============================================================
# SHAP ANALYSIS: CHILDHOOD vs ADULT ASTHMA ONSET
# =============================================================
#!pip install shap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shap
import joblib
import warnings
import json

warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300

# ============================================================================
# LOAD MODEL AND DATA
# ============================================================================

xgb_model = joblib.load('onset_xgb_model.pkl')
X_test = pd.read_csv('X_test_onset.csv')
y_test = pd.read_csv('y_test_onset.csv').values.ravel()
predictions = pd.read_csv('onset_predictions.csv')

# Class distribution
childhood_count = (y_test == 0).sum()
adult_count = (y_test == 1).sum()
total_count = len(y_test)



# Training set
y_train = pd.read_csv('y_train_onset.csv').values.ravel()
train_childhood = (y_train == 0).sum()
train_adult = (y_train == 1).sum()

# ============================================================================
# COMPUTE SHAP VALUES
# ============================================================================

explainer = shap.TreeExplainer(xgb_model)
shap_values = explainer.shap_values(X_test)

print(f"SHAP values computed: {shap_values.shape}")

# ============================================================================
# CREATE SHAP DATAFRAME FOR ANALYSIS
# ============================================================================

# Calculate mean absolute SHAP values for each feature
mean_abs_shap = np.abs(shap_values).mean(axis=0)
mean_shap = shap_values.mean(axis=0)

shap_df = pd.DataFrame({
    'Feature': X_test.columns,
    'Mean_Abs_SHAP': mean_abs_shap,
    'Mean_SHAP': mean_shap
})

# ============================================================================
# FIGURE: SHAP SUMMARY PLOT 
# ============================================================================

# Create SHAP summary plot (dot plot) with top 10 features
plt.figure(figsize=(10, 7))

shap.summary_plot(
    shap_values,
    X_test,
    max_display=10,
    plot_type="dot",
    show=False,
    color_bar_label="Feature value"
)

# Get current colorbar and resize it
cbar = plt.gcf().axes[-1]  # Get the colorbar axes
cbar.set_aspect(20)  # Make it narrower (higher number = narrower)

plt.title(
    "SHAP Summary Plot: Childhood vs Adult Onset Asthma (Top 10 Features)",
    fontsize=16,
    fontweight='bold',
    pad=20
)

plt.tight_layout()
plt.savefig("shap_summary.png", dpi=300, bbox_inches="tight")
plt.show()
plt.close()

# ============================================================================
# SAVE INTERPRETATION DATA
# ============================================================================

# Sort by absolute importance for export
interpretation_df = shap_df.sort_values('Mean_Abs_SHAP', ascending=False)
interpretation_df.to_csv('shap_feature_interpretation.csv', index=False)

# Identify strong predictors
childhood_predictors = interpretation_df[interpretation_df['Mean_SHAP'] < -0.01].head(10)
adult_predictors = interpretation_df[interpretation_df['Mean_SHAP'] > 0.01].head(10)

# Save detailed interpretation
with open('shap_interpretation.txt', 'w') as f:
    f.write("SHAP FEATURE INTERPRETATION\n")
    f.write("="*80 + "\n\n")
    
    f.write(f"Dataset Distribution:\n")
    f.write(f"  Test - Childhood: {childhood_count} ({childhood_count/total_count*100:.1f}%)\n")
    f.write(f"  Test - Adult: {adult_count} ({adult_count/total_count*100:.1f}%)\n")
    f.write(f"  Ratio: {childhood_count/adult_count:.2f}:1 (Childhood:Adult)\n\n")
    
    f.write("="*80 + "\n")
    f.write("Top 10 Features by Importance:\n\n")
    for i, row in interpretation_df.head(10).iterrows():
        direction = "→ ADULT" if row['Mean_SHAP'] > 0.01 else "→ CHILDHOOD" if row['Mean_SHAP'] < -0.01 else "→ NEUTRAL"
        f.write(f"{row['Feature']:<50} {direction:<15} SHAP={row['Mean_SHAP']:+.4f}\n")
    
    f.write("\n" + "="*80 + "\n")
    f.write("CHILDHOOD ONSET Predictors (SHAP < -0.01):\n")
    if len(childhood_predictors) > 0:
        for i, row in childhood_predictors.iterrows():
            f.write(f"  • {row['Feature']}: {row['Mean_SHAP']:.4f}\n")
    else:
        f.write("  • No strong individual predictors found\n")
    
    f.write("\nADULT ONSET Predictors (SHAP > 0.01):\n")
    if len(adult_predictors) > 0:
        for i, row in adult_predictors.iterrows():
            f.write(f"  • {row['Feature']}: {row['Mean_SHAP']:.4f}\n")
    else:
        f.write("  • No strong individual predictors found\n")

print("\nFile saved: shap_interpretation.txt")

# Save summary stats
summary_stats = {
    'test_samples': int(total_count),
    'childhood_count': int(childhood_count),
    'adult_count': int(adult_count),
    'childhood_percent': float(childhood_count/total_count*100),
    'adult_percent': float(adult_count/total_count*100),
    'childhood_to_adult_ratio': float(childhood_count/adult_count),
    'train_childhood': int(train_childhood),
    'train_adult': int(train_adult),
    'base_value': float(explainer.expected_value),
    'mean_prediction': float(predictions['xgb_proba'].mean()),
    'top_childhood_features': childhood_predictors['Feature'].tolist()[:5] if len(childhood_predictors) > 0 else [],
    'top_childhood_shap': [float(x) for x in childhood_predictors['Mean_SHAP'].tolist()[:5]] if len(childhood_predictors) > 0 else [],
    'top_adult_features': adult_predictors['Feature'].tolist()[:5] if len(adult_predictors) > 0 else [],
    'top_adult_shap': [float(x) for x in adult_predictors['Mean_SHAP'].tolist()[:5]] if len(adult_predictors) > 0 else []
}

with open('shap_summary_stats.json', 'w') as f:
    json.dump(summary_stats, f, indent=2)
